In [1]:
import sys

import os, copy, pickle, datetime, random
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Subset, TensorDataset

from utils import set_seed
from get_data_ours import get_loader


In [2]:
# Assign GPU number
GPU_NUM = 0  # Desired GPU index
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)  # Change allocation to the selected GPU
print('Current cuda device ', torch.cuda.current_device())  # Check current GPU index


Current cuda device  0


## Define functions

In [3]:

def get_erm_model(device, n_class=10, lr=0.0005, weight_decay=0, n_channel=3):
    print("Using a CNN model with Batch Normalization as feature extractor")

    # Feature Extractor 정의
    feature_extractor = nn.Sequential(
        nn.Conv2d(n_channel, 32, kernel_size=5, padding=2),  
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  
        nn.Conv2d(32, 48, kernel_size=5, padding=2),  
        nn.BatchNorm2d(48),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  

        nn.Conv2d(48, 64, kernel_size=5, padding=2), 
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  

        nn.Flatten(), 
    )

    dummy_input = torch.randn(1, n_channel, 32, 32)  
    with torch.no_grad():
        output_features = feature_extractor(dummy_input).view(-1).size(0)

    print(f"Feature Extractor Output Feature Size: {output_features}")

    label_predictor = nn.Sequential(
        nn.Linear(output_features, 128),  
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64, n_class)
    )

    feature_extractor = feature_extractor.to(device)
    label_predictor = label_predictor.to(device)
    models = {
        "G": feature_extractor,
        "C": label_predictor
    }

    optimizer_G = torch.optim.Adam(
        feature_extractor.parameters(), lr=lr, weight_decay=weight_decay,  betas=[0.5, 0.999]
    )
    optimizer_C = torch.optim.Adam(
        label_predictor.parameters(), lr=lr, weight_decay=weight_decay,  betas=[0.5, 0.999]
    )
    
    optimizers = [optimizer_G, optimizer_C]

    return models, optimizers



def prediction_batch(model_path, target_data_loader, device, models, latent=False):
    """
    Generate predictions in batches and return them.

    Args:
        model_path (str): Path to the model checkpoint.
        target_data_loader (DataLoader): DataLoader for target data.
        device (torch.device): Device to run on.
        models (dict): Dictionary of models to load (G: Feature Extractor, C: Classifier).
        latent (bool): If True, use inputs directly without the Feature Extractor.

    Returns:
        list: List of predictions for each batch, shape (batch_size, n_class).
    """
    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    models["G"].load_state_dict(checkpoint["G"])
    models["C"].load_state_dict(checkpoint["C"])

    # Set models to evaluation mode
    models["G"].eval()
    models["C"].eval()

    # Store predictions for each batch
    batch_predictions = []

    with torch.no_grad():
        for inputs, _ in target_data_loader:
            inputs = inputs.to(device)

            if not latent:
                features = models["G"](inputs)  # Extract features using the Feature Extractor
            else:
                features = inputs  # Use pre-extracted features directly

            predictions = models["C"](features)  # Generate predictions with the Classifier

            # Move predictions to CPU and store
            batch_predictions.append(predictions.cpu())

            # Free GPU memory
            del inputs, features, predictions
            torch.cuda.empty_cache()

    return batch_predictions



def get_classifier_predictions(classifiers, feature_extractor, unlabeled_loader, device):
    """
    Compute prediction probabilities on unlabeled data using multiple classifiers 
    and a feature extractor.

    Args:
        classifiers: List of trained classifiers (nn.Module)
        feature_extractor: Backbone model (ERM feature extractor)
        unlabeled_loader: DataLoader containing unlabeled images
        device: Computation device (cuda or cpu)

    Returns:
        stacked_probs: Tensor of shape (n_classifiers, N_unlabeled, num_classes)
    """
    print("Function called: get_classifier_predictions")

    feature_extractor.eval()
    for clf in classifiers:
        clf.eval()

    all_logits = []

    with torch.no_grad():
        # Extract features for the entire dataset
        all_features = []
        for x, _ in unlabeled_loader:
            x = x.to(device)
            feats = feature_extractor(x)  # (B, feature_dim)
            all_features.append(feats.cpu())

        features_all = torch.cat(all_features, dim=0).to(device)  # (N, D)

        # Run prediction with each classifier
        for clf in classifiers:
            logits = clf(features_all)  # (N, num_classes)
            all_logits.append(logits.cpu())

    # Stack predictions → (n_classifiers, N, num_classes)
    stacked_probs = torch.stack(all_logits)

    return stacked_probs


def mirror_descent_with_kl_ball(base_predictions, y_hat, nominal_weights, previous_weight,
                                num_base_model=5, lr=0.01, rho=1, threshold=1e-4,
                                max_iterations=20, rho_list=None):
    """
    Mirror Descent with KL-divergence and L2 ball projection to find optimal weights.
    """

    base_predictions = base_predictions.cpu()
    y_hat = y_hat.cpu()
    nominal_weights = nominal_weights.cpu()

    def run_mirror_descent(rho_value):
        weights = previous_weight.clone().detach().cpu()
        weights.requires_grad = True
        prev_weights = weights.clone().detach()
        criterion = torch.nn.CrossEntropyLoss()

        for i in range(max_iterations):
            y_tilda = torch.einsum('i,ijk->jk', weights, base_predictions)
            loss = criterion(y_tilda, y_hat)
            loss.backward()

            if weights.grad is None:
                raise RuntimeError("weights.grad is None. Autograd graph may have been broken.")

            with torch.no_grad():
                grad = weights.grad.clone()

                # Mirror descent update
                weights = weights * torch.exp(lr * grad)
                weights = weights / weights.sum()

                # L2 ball projection
                diff_from_nominal = torch.norm(weights - nominal_weights, p=2)
                if diff_from_nominal > rho_value:
                    weights = weights - nominal_weights
                    weights = weights * (rho_value / diff_from_nominal)
                    weights = weights + nominal_weights

                weights = weights.detach().requires_grad_()

            weights.grad = None
            weight_diff = torch.norm(weights - prev_weights, p=2).item()
            prev_weights.copy_(weights)

            if weight_diff < threshold:
                break

        return weights

    if rho_list is None:
        return run_mirror_descent(rho)
    return [run_mirror_descent(rho_val) for rho_val in rho_list]


def update_adv_weight(base_predictions, y_hat, nominal_weights, previous_weight, num_base_model, rho, lr_weight):
    """
    Mirror descent based weight update function on CPU.
    """
    adv_weight = mirror_descent_with_kl_ball(
        base_predictions, y_hat, nominal_weights, previous_weight,
        num_base_model=num_base_model, rho=rho, rho_list=None,
        lr=lr_weight
    )
    return adv_weight


def test_model(data_loader, models, device):
    """
    Evaluate model performance on a given dataset.
    
    Args:
        data_loader: DataLoader with test data
        models: dict containing feature extractor (G) and classifier (C)
        device: cuda or cpu
    
    Returns:
        avg_loss: average loss
        avg_accuracy: average accuracy
    """
    models["G"].eval()
    models["C"].eval()

    feature_extractor = models["G"]
    label_predictor = models["C"]
    loss_fn = torch.nn.CrossEntropyLoss()

    total_loss = 0.0
    correct_preds = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            features = feature_extractor(inputs)
            logits = label_predictor(features)

            loss = loss_fn(logits, labels)
            total_loss += loss.item() * inputs.size(0)

            preds = torch.argmax(logits, dim=1)
            correct_preds += (preds == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    avg_accuracy = correct_preds / total_samples

    return avg_loss, avg_accuracy

def train_model_batch(
    models, optimizers, loss_fn, dataloaders, base_predictions, nominal_weights, device,
    num_epochs=5, model_batch_size=128, max_outer_iteration=10, max_adv_iteration=1, threshold=1e-20,
    rho=0.1, lr_weight=0.01, scheduler=None, epsilon=0.1, patience=20, latent = True
):
    """
    Train model with adversarial weight updates and early stopping.
    """
    val_loader = dataloaders["valid"]
    target_loader = deepcopy(dataloaders["target_train"])
    test_data_loader = deepcopy(dataloaders["target_test"])

    best_val_accuracy = 0.0
    best_val_loss = float("inf")
    val_best_test_acc = 0.0
    best_test_loss = float("inf")
    adv_weight = nominal_weights.clone().cpu()
    no_improvement_count = 0

    val_loss, val_accuracy = test_model(val_loader, models, device)
    test_loss, test_accuracy = test_model(test_data_loader, models, device)
    print(f"Initial - Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")
    print(f"Initial - Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    for outer_iter in range(max_outer_iteration):
        print(f"\n--- Outer Iteration {outer_iter + 1}/{max_outer_iteration} ---")

        # Adversarial update
        _, adv_inputs = update_adv_loader(target_loader, models, device, epsilon, latent=latent)
        y_tilda = torch.einsum("i,ijk->jk", adv_weight, base_predictions)
        y_tilda_labels = torch.argmax(y_tilda, dim=1).long()

        adv_dataset = TensorDataset(adv_inputs, y_tilda_labels)
        adv_loader = DataLoader(adv_dataset, batch_size=target_loader.batch_size, shuffle=False)

        # Compute y_hat
        y_hat_list = []
        with torch.no_grad():
            for inputs, _ in adv_loader:
                if latent:
                    features = inputs.to(device)
                else:
                    inputs = inputs.to(device)
                    features = models["G"](inputs)
                y_hat_list.append(models["C"](features).cpu())
        y_hat_tensor = torch.cat(y_hat_list, dim=0)
        y_hat_labels = torch.argmax(y_hat_tensor, dim=1).long()

        # Weight update
        adv_weight = update_adv_weight(
            base_predictions,
            y_hat_labels,
            nominal_weights,
            previous_weight=adv_weight,
            num_base_model=num_base_model,
            rho=rho,
            lr_weight=lr_weight,
        )

        # Updated pseudo-labels
        y_tilda = torch.einsum("i,ijk->jk", adv_weight, base_predictions)
        y_tilda_labels = torch.argmax(y_tilda, dim=1).long()

        adv_dataset = TensorDataset(adv_inputs, y_tilda_labels)
        adv_loader = DataLoader(adv_dataset, batch_size=target_loader.batch_size, shuffle=True)

        # Training loop
        for epoch in range(num_epochs):
            models["G"].train()
            models["C"].train()
            total_loss, total_correct, total_samples = 0.0, 0, 0

            for inputs, labels in tqdm(adv_loader):
                inputs, labels = inputs.to(device), labels.to(device)

                if not latent:
                    optimizers[0].zero_grad()
                optimizers[1].zero_grad()

                features = inputs if latent else models["G"](inputs)
                logits = models["C"](features)
                loss = loss_fn(logits, labels)
                loss.backward()

                if not latent:
                    optimizers[0].step()
                optimizers[1].step()

                total_loss += loss.item() * inputs.size(0)
                total_correct += (torch.argmax(logits, dim=1) == labels).sum().item()
                total_samples += inputs.size(0)

            train_loss = total_loss / total_samples
            train_accuracy = total_correct / total_samples
            val_loss, val_accuracy = test_model(val_loader, models, device)
            test_loss, test_accuracy = test_model(test_data_loader, models, device)

            print(f"Epoch {epoch+1}/{num_epochs} - "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f} | "
                  f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")

            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                val_best_test_acc = test_accuracy
                best_val_loss = val_loss
                best_test_loss = test_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    print(f"Early stopping at Epoch {epoch+1}")
                    return models, val_best_test_acc, best_val_loss, best_test_loss, adv_loader

            if scheduler is not None:
                scheduler.step(train_loss)

    print("Training completed.")
    return models, val_best_test_acc, best_val_loss, best_test_loss, adv_loader


def update_adv_loader(target_loader, models, device, epsilon, latent=False):
    """
    Generate adversarial inputs (x_adv or z_adv) and return a new DataLoader.
    """
    adv_inputs = []
    adv_labels = []

    for inputs, labels in target_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        if not latent:
            inputs.requires_grad = True
            features = models["G"](inputs)
            logits = models["C"](features)
            loss = torch.nn.CrossEntropyLoss()(logits, labels)
            loss.backward()
            adv_input = inputs + epsilon * inputs.grad.sign()
        else:
            with torch.no_grad():
                z = models["G"](inputs)
            z.requires_grad = True
            logits = models["C"](z)
            loss = torch.nn.CrossEntropyLoss()(logits, labels)
            loss.backward()
            adv_input = z + epsilon * z.grad.sign()

        adv_inputs.append(adv_input.detach().cpu())
        adv_labels.append(labels.detach().cpu())

    adv_inputs = torch.cat(adv_inputs, dim=0)
    adv_labels = torch.cat(adv_labels, dim=0)
    adv_dataset = TensorDataset(adv_inputs, adv_labels)
    adv_loader = DataLoader(adv_dataset, batch_size=target_loader.batch_size, shuffle=False)

    return adv_loader, adv_inputs

def save_and_print_results(base_model_results, timestamp, dataset, type_base_model, max_outer_iteration, latent, seed, max_adv_iteration=1, u=0):
    """
    Print and save final results as a CSV file.
    """
    print("---- Final Results ----")
    for result in base_model_results:
        print(f"num_base_model={result[0]}, rho={result[1]}, epsilon={result[2]} | "
              f"Val-Best Test Loss={result[3]:.4f}, Val-Best Test Acc={result[4]:.4f} | "
              f"Last Test Loss={result[5]:.4f}, Last Test Acc={result[6]:.4f}")

    results_df = pd.DataFrame(base_model_results, columns=[
        "num_base_model", "e2", "e1", "val_best_test_loss", "val_best_test_acc", "last_test_loss", "last_test_accuracy"
    ])

    csv_save_path = os.path.join(
        "./Ours_results", f"{dataset}",
        f"{timestamp}_{dataset}_{type_base_model}_max_iter_{max_outer_iteration}_latent_{latent}_u{u}.csv"
    )

    os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)
    results_df.to_csv(csv_save_path, index=False)

    print(f"Results saved to {csv_save_path}")



def split_source_loader(s_train_loader, K=100, num_splits=10, batch_size=128, replacement=True, seed=11):
    dataset = s_train_loader.dataset
    targets = dataset.targets if hasattr(dataset, "targets") else [y for _, y in dataset]
    label_to_indices = defaultdict(list)
    for idx, label in enumerate(targets):
        label = int(label)
        label_to_indices[label].append(idx)

    random.seed(seed)
    split_loaders = []
    for split_idx in range(num_splits):
        selected_indices = []
        for label, indices in label_to_indices.items():
            chosen = random.choices(indices, k=K) if replacement else random.sample(indices, k=K)
            selected_indices.extend(chosen)

        subset = Subset(dataset, selected_indices)
        loader = DataLoader(subset, batch_size=batch_size, shuffle=True, drop_last=True)
        split_loaders.append(loader)

    return split_loaders


def train_classifiers(split_loaders, ERM_feature_map, valid_loader, n_class, device, 
                      n_epochs=20, lr=1e-3, weight_decay=0, patience=10):
    """
    Train linear classifiers on split feature sets with early stopping.
    """
    classifiers = []
    ERM_feature_map.eval()  # keep feature extractor fixed

    for split_idx, loader in enumerate(split_loaders):
        print(f"\n--- Split {split_idx+1}/{len(split_loaders)} ---")

        # Get feature dimension
        with torch.no_grad():
            dummy_input, _ = next(iter(loader))
            dummy_input = dummy_input.to(device)
            feature_dim = ERM_feature_map(dummy_input).view(dummy_input.size(0), -1).size(1)

        classifier = nn.Linear(feature_dim, n_class).to(device)
        optimizer = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        best_val_acc, no_improve = 0.0, 0

        for epoch in range(n_epochs):
            classifier.train()
            total_loss, correct, total = 0.0, 0, 0

            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.no_grad():
                    features = ERM_feature_map(inputs).view(inputs.size(0), -1)

                outputs = classifier(features)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_loss = total_loss / total
            train_acc = correct / total

            # Validation
            classifier.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    features = ERM_feature_map(inputs).view(inputs.size(0), -1)
                    outputs = classifier(features)
                    preds = torch.argmax(outputs, dim=1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.size(0)
            val_acc = val_correct / val_total

            print(f"Split {split_idx+1}, Epoch {epoch+1}/{n_epochs} | "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                  f"Val Acc: {val_acc:.4f}")

            if val_acc > best_val_acc:
                best_val_acc, no_improve = val_acc, 0
            else:
                no_improve += 1

            if no_improve >= patience:
                
                break

        classifiers.append(classifier)

    return classifiers



## Parameter setting

In [ ]:
seed = 0
set_seed(seed)

data_path = "./data"
dataset = "s2m"
partition = 2000
num_val = 100
target_sparse = True
batch_size = 100
ERM_path = "./Digit_ERM"  # path for ERM feature map

# Common training parameters
loss_fn = torch.nn.CrossEntropyLoss()
init_erm = True
n_class = 10  # number of classes (e.g., MNIST/USPS = 10)

# ERM feature extractor parameters
erm_model_lr = 1e-3
erm_weight_decay = 1e-6

# Classifier parameters
clf_lr = 1e-2
clf_weight_decay = 0
clf_epochs = 20
clf_patience = 5

# Target data setting
u = 100  # number of unlabeled target samples

# Base model parameters
num_base_model = 10
base_model_lr = 1e-3
base_model_weight_decay = 0
base_model_epochs = 2
early_stop_threshold = 1e-20
max_outer_iteration = 150

# DRO parameters
rho_values = [0, 0.2, 0.4, 0.6, 0.9] 
epsilon_values = [0, 0.2, 0.4, 0.6, 0.8, 1]

# General settings
latent = True
type_base_model = "ERM"

# Path to save/load model
default_path = f"{ERM_path}/{dataset}_u{u}_ERM_model.pth"


## SHVN -> MNIST

### DATA Loading

In [5]:
s_train_loader, s_test_loader, t_train_loader, t_val_loader, t_test_loader, target_loader = get_loader(
    dset=dataset,
    data_path=data_path,
    batch_size=batch_size,
    num_workers=4,
    num_val=num_val,
    target_sparse=target_sparse,
    u=u
)

# Example: split source loader
split_loaders = split_source_loader(
    s_train_loader, K=partition, num_splits=num_base_model, batch_size=128, replacement=True, seed=seed
)


dataloaders = {
    "source": s_train_loader,
    "target_train": t_train_loader,
    "target_test": t_test_loader,
    "valid": t_val_loader
}

# Check input/output shapes
n_channel = None
print("Source DataLoader:")
for batch_idx, (inputs, targets) in enumerate(dataloaders["source"]):
    print(f"Source Batch {batch_idx} | Inputs: {inputs.shape} | Targets: {targets.shape}")
    break  

print("\nTarget DataLoader:")
for batch_idx, (inputs, targets) in enumerate(dataloaders["target_train"]):
    n_channel = inputs.shape[1]
    print(f"Target Batch {batch_idx} | Inputs: {inputs.shape} | Targets: {targets.shape}")
    break 

print(f"Num Channels: {n_channel}")



# ================== ERM model ==================
# NOTE: Requires a pre-trained ERM model saved at `default_path`.
checkpoint = torch.load(default_path, map_location=device)
models, optimizers = get_erm_model(
    device, n_class=n_class, 
    lr=erm_model_lr, 
    weight_decay=erm_weight_decay, 
    n_channel=n_channel
)
models["G"].load_state_dict(checkpoint["G"])
ERM_feature_map = copy.deepcopy(models["G"])


# ================== Base Classifier Training ==================
classifiers = train_classifiers(
    split_loaders=split_loaders,
    ERM_feature_map=ERM_feature_map,
    valid_loader=t_val_loader,
    n_class=n_class,
    device=device,
    n_epochs=clf_epochs,
    lr=clf_lr,
    weight_decay=clf_weight_decay,
    patience=clf_patience
)



No noise perturbed
Source DataLoader:
Source Batch 0 | Inputs: torch.Size([100, 3, 32, 32]) | Targets: torch.Size([100])

Target DataLoader:
Target Batch 0 | Inputs: torch.Size([100, 3, 32, 32]) | Targets: torch.Size([100])
Num Channels: 3
Using a CNN model with Batch Normalization as feature extractor
Feature Extractor Output Feature Size: 1024

--- Split 1/10 ---
Split 1, Epoch 1/20 | Train Loss: 0.6012, Train Acc: 0.8313 | Val Acc: 0.6300
Split 1, Epoch 2/20 | Train Loss: 0.2503, Train Acc: 0.9292 | Val Acc: 0.5700
Split 1, Epoch 3/20 | Train Loss: 0.2027, Train Acc: 0.9425 | Val Acc: 0.6100
Split 1, Epoch 4/20 | Train Loss: 0.1774, Train Acc: 0.9473 | Val Acc: 0.6500
Split 1, Epoch 5/20 | Train Loss: 0.1485, Train Acc: 0.9577 | Val Acc: 0.5600
Split 1, Epoch 6/20 | Train Loss: 0.1287, Train Acc: 0.9647 | Val Acc: 0.5900
Split 1, Epoch 7/20 | Train Loss: 0.1282, Train Acc: 0.9632 | Val Acc: 0.5800
Split 1, Epoch 8/20 | Train Loss: 0.1140, Train Acc: 0.9673 | Val Acc: 0.5800
Split 1,

### Model training

In [6]:
print("------------------------------------ Starting Main Code ------------------------------------\n")
max_outer_iteration = 150
# ================== Prediction ==================
train_data_loader = dataloaders["target_train"]
test_data_loader = dataloaders["target_test"]

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')

base_predictions = get_classifier_predictions(
    classifiers=classifiers, 
    feature_extractor=ERM_feature_map, 
    unlabeled_loader=train_data_loader, 
    device=device
)
nominal_weights = torch.full((num_base_model,), 1 / num_base_model).to(device)


base_model_results = []
for rho in rho_values:  # rho 값 반복
    for epsilon in epsilon_values:  # epsilon 값 반복
        print(f"------------------------ Starting model training with num_base_model={num_base_model}, rho={rho}, epsilon={epsilon} ------------------------")
        print(f"Base model predictions generated with {num_base_model} base models and nominal weights: {nominal_weights}\n")

        # model 초기화
        checkpoint = torch.load(default_path, map_location=device)
        models, optimizers = get_erm_model(
            device, n_class=n_class,
            weight_decay=0, 
            n_channel=n_channel
        )
        models["G"].load_state_dict(checkpoint["G"])
        models["C"].load_state_dict(checkpoint["C"])
                
        models_copy = copy.deepcopy(models)
        models_copy["G"] = models["G"].to(device)
        models_copy["C"] = models["C"].to(device)

        optimizer_G = torch.optim.Adam(models_copy["G"].parameters(), lr=base_model_lr, weight_decay=base_model_weight_decay)
        optimizer_C = torch.optim.Adam(models_copy["C"].parameters(), lr=base_model_lr, weight_decay=base_model_weight_decay)
        optimizers_copy = [optimizer_G, optimizer_C]
        
        # 학습 및 평가
        trained_model, val_best_test_acc, best_val_loss, best_test_loss, _ = train_model_batch(
            models_copy, optimizers_copy, loss_fn, dataloaders, base_predictions, nominal_weights, device,
            num_epochs=base_model_epochs, model_batch_size=batch_size, max_outer_iteration=max_outer_iteration,
            threshold=1e-10, rho=rho, lr_weight=0.05, scheduler=None, epsilon=epsilon
        )            
        
        # 마지막 테스트 결과
        test_loss, test_accuracy = test_model(test_data_loader, trained_model, device)

        print(f"Best Val-Test Results for num_base_model={num_base_model}, rho={rho}, epsilon={epsilon}: Test Loss: {best_test_loss:.4f}, Test Accuracy: {val_best_test_acc:.4f}\n")
        print(f"Last Test Results for num_base_model={num_base_model}, rho={rho}, epsilon={epsilon}: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}\n")

        
        # 결과 저장
        base_model_results.append(
            (num_base_model, rho, epsilon, best_test_loss, val_best_test_acc, test_loss, test_accuracy)
        )
        


------------------------------------ Starting Main Code ------------------------------------

Function called: get_classifier_predictions
------------------------ Starting model training with num_base_model=10, rho=0, epsilon=0 ------------------------
Base model predictions generated with 10 base models and nominal weights: tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], device='cuda:0')

Using a CNN model with Batch Normalization as feature extractor
Feature Extractor Output Feature Size: 1024
Initial - Test Loss: 1.8787, Test Acc: 0.6250
Initial - Val Loss: 1.7445, Val Acc: 0.7100

--- Outer Iteration 1/150 ---


100%|██████████| 10/10 [00:00<00:00, 199.94it/s]

Epoch 1/2 - Train Loss: 0.7033, Train Acc: 0.7910 | Val Loss: 1.8410, Val Acc: 0.6400 | Test Loss: 1.7819, Test Acc: 0.6608


Epoch 2/2 - Train Loss: 0.2533, Train Acc: 0.9060 | Val Loss: 2.0695, Val Acc: 0.6300 | Test Loss: 1.9086, Test Acc: 0.6471

--- Outer Iteration 2/150 ---


100%|██████████| 10/10 [00:00<00:00, 180.69it/s]

...

100%|██████████| 10/10 [00:00<00:00, 156.31it/s]

Epoch 1/2 - Train Loss: 0.6412, Train Acc: 0.7620 | Val Loss: 0.4456, Val Acc: 0.8600 | Test Loss: 0.3263, Test Acc: 0.9072


Epoch 2/2 - Train Loss: 0.4342, Train Acc: 0.8400 | Val Loss: 0.4853, Val Acc: 0.8300 | Test Loss: 0.3888, Test Acc: 0.8737

--- Outer Iteration 30/150 ---


100%|██████████| 10/10 [00:00<00:00, 331.11it/s]


Epoch 1/2 - Train Loss: 0.6987, Train Acc: 0.7440 | Val Loss: 0.4958, Val Acc: 0.8600 | Test Loss: 0.3471, Test Acc: 0.8945
Early stopping at Epoch 1
Best Val-Test Results for num_base_model=10, rho=0.9, epsilon=1: Test Loss: 0.3700, Test Accuracy: 0.9001

Last Test Results for num_base_model=10, rho=0.9, epsilon=1: Test Loss: 0.3471, Test Accuracy: 0.8945



In [7]:
# 모든 결과 출력
save_and_print_results(
    base_model_results=base_model_results,
    timestamp=timestamp,
    dataset=dataset,
    type_base_model=type_base_model,
    max_outer_iteration=max_outer_iteration,
    latent=latent,
    seed = seed,
    u=u
)

torch.cuda.empty_cache()

---- Final Results ----
num_base_model=10, rho=0, epsilon=0 | Val-Best Test Loss=2.1299, Val-Best Test Acc=0.6373 | Last Test Loss=2.3344, Last Test Acc=0.6466
num_base_model=10, rho=0, epsilon=0.2 | Val-Best Test Loss=0.3902, Val-Best Test Acc=0.8946 | Last Test Loss=0.4152, Last Test Acc=0.8959
num_base_model=10, rho=0, epsilon=0.4 | Val-Best Test Loss=0.2916, Val-Best Test Acc=0.9130 | Last Test Loss=0.2716, Last Test Acc=0.9239
num_base_model=10, rho=0, epsilon=0.6 | Val-Best Test Loss=0.4389, Val-Best Test Acc=0.8785 | Last Test Loss=0.2853, Last Test Acc=0.9122
num_base_model=10, rho=0, epsilon=0.8 | Val-Best Test Loss=0.3124, Val-Best Test Acc=0.9074 | Last Test Loss=0.3172, Last Test Acc=0.8967
num_base_model=10, rho=0, epsilon=1 | Val-Best Test Loss=0.3643, Val-Best Test Acc=0.8937 | Last Test Loss=0.4081, Last Test Acc=0.8712
num_base_model=10, rho=0.2, epsilon=0 | Val-Best Test Loss=1.8423, Val-Best Test Acc=0.6561 | Last Test Loss=2.1443, Last Test Acc=0.6450
num_base_model